In [64]:
import pandas as pd
from sklearn.feature_selection import RFECV
from sklearn import linear_model
from sklearn.cross_validation import ShuffleSplit
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

import matplotlib.pyplot as plt
import scipy.stats
from sklearn.metrics import roc_curve, auc
import random
%matplotlib inline
%pylab inline 

from sklearn.naive_bayes import GaussianNB
from sklearn import svm
import sys
import os.path
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [65]:
def addClassLabel(df,target_genes,random_genes):
    targets=df.loc[df["ref_gene_name"].isin(target_genes)]
    unknown=df.loc[df["ref_gene_name"].isin(random_genes)]
    targets["class_label"]=1
    unknown["class_label"]=0
    #print(len(targets),'len targets')
    #print(len(unknown),'len unknown')
    train=targets.append(unknown)
    train=train.fillna(value=0)
    return train

In [66]:
#build a training set more than one dataset with known target genes (do this for all 54 motifs with chip data TG predictions)
#first test with unc3, ase,aiy


def import_data(motif,r):
    #r is the random seed used to draw genes
    #import the data
    p="training_data/%s_all_info_normed.txt" %motif
    #p="/Users/lori/Google\ Drive/Hobert_Lab_ELN/TargetOrtho_updates/TargetOrtho2.0/%s_TargetOrtho2.0_Results/%s__all_info_normed.txt" %(motif,motif)
    df=pd.read_csv(p,sep='\t')
    #import the target genes
    target_genes=list(pd.read_csv("training_data/target_genes/%s_target_genes.txt" %motif,sep="\t")["target_genes"].values)
    #print('len df',len(df), 'len target genes',len(target_genes))
    #label the data as training vs random (class_label)
    #get list of random ref gene names that is same size as target gene list (and excludes target genes)
    t_genes=list(df.loc[df["ref_gene_name"].isin(target_genes)]["ref_gene_name"].values)
    r_genes=list(df.loc[~df["ref_gene_name"].isin(target_genes)]["ref_gene_name"].values)
    import random
    random.seed(r)
    random_genes=random.sample(r_genes,len(t_genes))
    #print(random_genes)
    #print(len(target_genes),len(t_genes))
    train=addClassLabel(df,t_genes,random_genes)
    
    #make sure conservation is normed between 1 and 0 (for 8 species, wasn't properly normed)
    c=list(train["conservation_upstream"].values)
    c_normed=normData(c)
    train["conservation_upstream"]=c_normed
    
    c=list(train["conservation_intron"].values)
    c_normed=normData(c)
    train["conservation_intron"]=c_normed
    #,len(target_genes),len(t_genes)
    
    return train


In [67]:
def fit_model(X_train,y_train,X_test,y_test,model):
    """perform supervised learning: train classifier, predict test data
    returns evaulation results for a given model"""
    probs =model.fit(X_train, y_train).predict_proba(X_test)
    y_preds=model.fit(X_train, y_train).predict(X_test)
    accur=model.fit(X_train, y_train).score(X_test,y_test)
    fpr, tpr, thresholds = roc_curve(y_test, probs[:, 1])
    roc_auc = auc(fpr, tpr)
    recall=get_recall(y_preds,y_test)
    return recall,accur,roc_auc,fpr,tpr,thresholds, probs,y_preds

In [68]:
def adjust_probs(probs):
    """takes a list of classifier probabilities of form (class 0 probability, class 1 probability) 
    and transforms class 0 probabilites by multiplying by -1. This results is high probabilty of class 1
    having the largest values and high probability of class 0 having the lowerst values"""
    plist=[]
    for i in range(len(probs)):
        ps= str(probs[i]).split()
        B=float(ps[-1][:-1])
        A=float(ps[-2])
        if A>B:plist.append(float(A*-1))
        else:plist.append(float(B))
    return plist

In [69]:
def rankData(data):
    """takes a list of numbers of returns the list in place with elements replaced by rank order"""
    uni=list(set(data))
    uni.sort()
    uni.reverse()
    uniDic={}
    for i in range(len(uni)):
        uniDic[uni[i]]=i+1
    rankedList=[uniDic[n] for n in data]
    return rankedList

In [70]:
def get_class_list(ranked_data,y_test):
    """get list of ranks for each class label"""
    tlist=[]
    rlist=[]
    for i in range(len(ranked_data)):
        #print n[i],y_test[i]
        if y_test[i]==0:rlist.append(ranked_data[i])
        else:tlist.append(ranked_data[i])
    return tlist,rlist

In [71]:
def shuffle_for_cv(random_state,X,y):
    """shuffle the dataset and split into halves for testing and training set generation """
    X, y = shuffle(X, y, random_state=random_state)
    n_samples, n_features = X.shape
    half = int(n_samples / 2)
    X_train, X_test = X[:half], X[half:]
    y_train, y_test = y[:half], y[half:]
    
    #train on 90% test on 10%
    #200 samples, ratio =20
    #ratio=int(n_samples/10)
    #X_train, X_test = X[ratio:], X[:ratio]
    #y_train, y_test = y[ratio:], y[:ratio]
    
    return X_train,y_train,X_test,y_test

In [72]:
def model_data(train,r,feature_set):
    #print(r,'r')
    Xb=train[feature_set]
    yb=array(train["class_label"])
    random_state=np.random.RandomState(r)
    Xb_train,yb_train,Xb_test,yb_test=shuffle_for_cv(random_state,Xb,yb)
    #fit the balanced shuffled cv set to model
    
    #swap train and test and get stats again, return both.
    recall,accur,roc_auc,fpr,tpr,thresholds, probs,y_preds=fit_model(Xb_train,yb_train,Xb_test,yb_test,model)
    recall2,accur2,roc_auc2,fpr2,tpr2,thresholds2, probs2,y_preds2=fit_model(Xb_test,yb_test,Xb_train,yb_train,model)
    
    #for each cross validation, get ranks of test data (ranked by class assignment probability)
    adj_probs=adjust_probs(probs)
    ranked_data=rankData(adj_probs)
    
    adj_probs2=adjust_probs(probs2)
    ranked_data2=rankData(adj_probs2)

    #perform ranksums test to assess difference between ranks of random genes and validated target genes
    tlist,rlist=get_class_list(ranked_data,yb_test)
    tlist2,rlist2=get_class_list(ranked_data2,yb_train)
    
    rs=get_ranksums(tlist,rlist)
    ranksum_p=log10(float(rs[1]))*-1
    
    rs2=get_ranksums(tlist2,rlist2)
    ranksum_p2=log10(float(rs2[1]))*-1
    
    
    return [fpr,tpr,rs[0],ranksum_p,recall,accur,roc_auc], [fpr2,tpr2,rs2[0],ranksum_p2,recall2,accur2,roc_auc2]

In [73]:
def model_data_by_motifs(train,test,r,feature_set):
    Xb_train=train[feature_set]
    yb_train=array(train["class_label"])
    #random_state=np.random.RandomState(r)
    #Xb_train,yb_train,Xb_test,yb_test=shuffle_for_cv(random_state,Xb,yb)
    #fit the balanced shuffled cv set to model
    Xb_test=test[feature_set]
    yb_test=array(test["class_label"])
    
    recall,accur,roc_auc,fpr,tpr,thresholds, probs,y_preds=fit_model(Xb_train,yb_train,Xb_test,yb_test,model)
    #for each cross validation, get ranks of test data (ranked by class assignment probability)
    adj_probs=adjust_probs(probs)
    ranked_data=rankData(adj_probs)

    #perform ranksums test to assess difference between ranks of random genes and validated target genes
    tlist,rlist=get_class_list(ranked_data,yb_test)
    rs=get_ranksums(tlist,rlist)
    ranksum_p=log10(float(rs[1]))*-1
    
    
    return fpr,tpr,rs[0],ranksum_p,recall,accur,roc_auc

In [74]:
def get_recall(y_pred,y_test):
    """get true positive count from model class predictions,return recall (TP/P)"""
    tp=0
    for p in range(len(y_pred)):
        if y_test[p]==1 and y_pred[p]==1:tp+=1
    return float(tp)/sum(y_test)

In [75]:

def get_ranksums(tlist,rlist):
    """get wilcoxon rank sums test results for two lists of ranks
    returns z score, p value"""
    return scipy.stats.ranksums(rlist,tlist)

In [76]:
def normData(l):
    maxL=float(max(l))
    minL=min(l)
    return [i/(maxL-minL) for i in l]

In [77]:
def getHeaders(df):
    #print("getHeaders")
    headers=[]
    for i in list(df.columns.values):
        if 'avg_species_' not in i:
            #headers.append(i)
            if 'dist' not in i:
                #headers.append(i)
                if 'score' in i:
                    headers.append(i)
                if 'count' in i:
                    headers.append(i)
                if 'conservation' in i:
                    headers.append(i)
    return headers

In [78]:
#Support Vector Machine Classifier
SVM1=svm.SVC(kernel='rbf',probability=True,class_weight='balanced')
#SVM2=svm.SVC(kernel='poly',probability=True,class_weight='balanced')
#SVM3=svm.SVC(kernel='linear',probability=True,class_weight='balanced')

#linear and quadratic discriminant analysis (vary the solver and the shrinkage params)
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
qda = QuadraticDiscriminantAnalysis(store_covariances=True)

#Stochastic Gradient Descent
sgd = SGDClassifier(loss="log", penalty="l2")

#K nearest neighbor
KNN = KNeighborsClassifier(n_neighbors=3)

#RadiusNeighborsClassifier
#RN = NearestNeighbors(radius=1.6)

#Gaussian Process Classification (GPC)
GPC = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0),optimizer=None)


#decision tree classifier
DTC = tree.DecisionTreeClassifier()

#random forest
RFC= RandomForestClassifier(n_estimators=10)

ABC = AdaBoostClassifier(n_estimators=100)


GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)



#Gaussian Naive Bayes Classifier
NB=GaussianNB()



#Logistic Regression Classifier
LR=linear_model.LogisticRegression(class_weight='balanced')

VC = VotingClassifier(estimators=[('lr', LR), ('rf', RFC), ('gnb', NB)], voting='soft')

#supervised neural network. multi-layer perceptron (MLP), alpha param for L2 regularization
MLP = MLPClassifier(solver='lbfgs', alpha=1e-5,activation="logistic", random_state=1,validation_fraction=0.5)

all_models=[lda,qda,sgd,KNN,GPC,DTC,RFC,ABC,GBC,VC,MLP,LR,NB,SVM1]



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


'\nalphas = np.logspace(-5, 3, 5)\nsolvers=[\'lbfgs\', \'sgd\', \'adam\']\nactivations=[\'identity\', \'logistic\', \'tanh\', \'relu\']\n\nnames = []\nclassifiers = []\nfor i in alphas:\n    for s in solvers:\n        for a in activations:\n            classifiers.append(MLPClassifier(activation=a,learning_rate="adaptive",solver=s,alpha=i, random_state=1,validation_fraction=0.5))\n            names.append("%s_%s_%s" %(i,s,a))\n'

In [79]:
motifs=["COE","ASE","AIY"]
train=import_data(motifs[0],r)
#print(motifs[0],len(train))
for motif in motifs[1:]:
    t_df=import_data(motif,r)
    train=train.append(t_df)
print(len(train))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


200


In [106]:

motifs=["COE","ASE","AIY"]


models=all_models

model_names=['LDA','QDA','SGD','KNN','GPC','DTC','RFC','ABC','GBC','VC','MLP','LR','NB','SVM1']



motif_set_name="3_OH_lab_motifs"
print("model","m_recall","m_accur","m_roc_auc","m_ranksum_p","m_ranksum_z")
outfile=open("050919_Results.csv",'w')
outfile.write("%s,%s,%s,%s,%s,%s,%s\n" %("motif_set","model","k","i","recall","ranksum_z","ranksum_p"))
#r is the random seed used to draw genes
r=0
train=import_data(motifs[0],r)
#print(train)
feature_set=getHeaders(train)

for i in range(len(models)):
    model=models[i]
    model_name=model_names[i]
    #print(model)
    recalls=[]
    accurs=[]
    roc_aucs=[]
    ranksum_zs=[]
    ranksum_ps=[]
    fprs=[]
    tprs=[]
    print(model_name)
    
    for r in range(10):
        
        train=import_data(motifs[0],r)
        
        for motif in motifs[1:]:
            
            t_df=import_data(motif,r)
            train=train.append(t_df)
        
        
        results1,results2=model_data(train,r,feature_set)

      
        outfile.write("%s,%s,%s,%s,%s,%s,%s\n" %(motif_set_name,model_name,r+1,1,results1[4],results1[2],results1[3]))
        outfile.write("%s,%s,%s,%s,%s,%s,%s\n" %(motif_set_name,model_name,r+1,2,results2[4],results2[2],results2[3]))
        
        print("%s,%s,%s,%s,%s,%s,%s\n" %(motif_set_name,model_name,r+1,1,results1[4],results1[2],results1[3]))
        print("%s,%s,%s,%s,%s,%s,%s\n" %(motif_set_name,model_name,r+1,2,results2[4],results2[2],results2[3]))
       
        fprs.append(fpr)
        tprs.append(tpr)
        ranksum_zs.append(ranksum_z)
        ranksum_ps.append(ranksum_p)
        recalls.append(recall)
        accurs.append(accur)
        roc_aucs.append(roc_auc)
        m_recall=sum(recalls)/len(recalls)
    print('len recalls',len(recalls))
    #get mean stats over all cross validations
    m_accur=sum(accurs)/len(accurs)
    m_roc_auc=sum(roc_aucs)/len(roc_aucs)

    m_ranksum_p=sum(ranksum_ps)/len(ranksum_ps)
    m_ranksum_z=sum(ranksum_zs)/len(ranksum_zs)

    #get median auc_roc (if list size is even, use value adjacent to median (can't use mean of medians for plotting))
    #sort auc_rocs, get value at middle,get index of value in unsorted list, get fpr list at same index
    #make copy of aucs list, then sort the copy
    sorted_roc_aucs=sorted(roc_aucs)
    mid=int((len(roc_aucs)/2))
    med_auc=sorted_roc_aucs[mid]
    i=roc_aucs.index(med_auc)
    med_fpr=fprs[i]
    med_tpr=tprs[i]
    #get standard error of the mean for results
    sem_recalls=scipy.stats.sem(recalls)
    sem_accurs=scipy.stats.sem(accurs)
    sem_roc_aucs=scipy.stats.sem(roc_aucs)
    sem_ranksum_ps=scipy.stats.sem(ranksum_ps)
    sem_ranksum_zs=scipy.stats.sem(ranksum_zs)
    
    print(motif_set_name,model_name,m_recall,m_accur,m_roc_auc,m_ranksum_p,m_ranksum_z,len(train))
    #print(sem_recalls,sem_accurs,sem_roc_aucs,sem_ranksum_ps,sem_ranksum_zs)
outfile.close()

model m_recall m_accur m_roc_auc m_ranksum_p m_ranksum_z


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


LDA


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:510: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


3_OH_lab_motifs,LDA,1,1,0.535714285714,3.07271909592,2.67342236055

3_OH_lab_motifs,LDA,1,2,0.659090909091,3.88863885585,3.99650504382

3_OH_lab_motifs,LDA,2,1,0.630434782609,2.95658425271,2.5071457333

3_OH_lab_motifs,LDA,2,2,0.648148148148,4.27753768492,4.72360259911

3_OH_lab_motifs,LDA,3,1,0.733333333333,4.04973728002,4.29009293708

3_OH_lab_motifs,LDA,3,2,0.618181818182,4.79109209433,5.7802165433

3_OH_lab_motifs,LDA,4,1,0.6,3.48137903711,3.30203956747

3_OH_lab_motifs,LDA,4,2,0.58,3.12290040358,2.74695695003

3_OH_lab_motifs,LDA,5,1,0.63829787234,5.4663066548,7.33770586314

3_OH_lab_motifs,LDA,5,2,0.622641509434,4.26461700485,4.6984374758

3_OH_lab_motifs,LDA,6,1,0.674418604651,3.73880285978,3.73306626474

3_OH_lab_motifs,LDA,6,2,0.578947368421,3.27580399539,2.97731741558

3_OH_lab_motifs,LDA,7,1,0.711111111111,4.7252708725,5.6386333036

3_OH_lab_motifs,LDA,7,2,0.509090909091,3.27373971738,2.97414414048

3_OH_lab_motifs,LDA,8,1,0.603773584906,3.30119340618,3.01648876329

3_OH_lab

...

3_OH_lab_motifs,VC,9,1,0.551020408163,5.85057639664,8.30991703788

3_OH_lab_motifs,VC,9,2,0.647058823529,6.20223154835,9.25439367061



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


3_OH_lab_motifs,VC,10,1,0.695652173913,5.49820931206,7.41604514708

3_OH_lab_motifs,VC,10,2,0.537037037037,6.21747191389,9.2965121113

len recalls 10
3_OH_lab_motifs VC 15.8931131786 0.73 0.842592592593 8.40138297641 -5.8855045592 200
MLP


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


3_OH_lab_motifs,MLP,1,1,0.714285714286,4.10737479149,4.39774289968

3_OH_lab_motifs,MLP,1,2,0.772727272727,3.31923102339,3.04447715937

3_OH_lab_motifs,MLP,2,1,0.739130434783,3.90407441089,4.02417002645

3_OH_lab_motifs,MLP,2,2,0.759259259259,5.09708209181,6.46227057178

3_OH_lab_motifs,MLP,3,1,0.822222222222,4.54166430634,5.25327348325

3_OH_lab_motifs,MLP,3,2,0.727272727273,4.6836995745,5.55014707014

3_OH_lab_motifs,MLP,4,1,0.76,4.75673571407,5.70608862199

3_OH_lab_motifs,MLP,4,2,0.8,4.52923965818,5.22770526043

3_OH_lab_motifs,MLP,5,1,0.68085106383,5.28674383354,6.90478596196

3_OH_lab_motifs,MLP,5,2,0.716981132075,5.71147896843,7.95078763758

3_OH_lab_motifs,MLP,6,1,0.720930232558,4.2992751693,4.76609660197

3_OH_lab_motifs,MLP,6,2,0.684210526316,4.90500300692,6.02953267218

3_OH_lab_motifs,MLP,7,1,0.777777777778,6.12483790512,9.0420290912

3_OH_lab_motifs,MLP,7,2,0.618181818182,5.39041164047,7.15306646241

3_OH_lab_motifs,MLP,8,1,0.88679245283,5.35235332592,7.06139188266

3_OH_l

In [108]:
#run on individidual motifs
motifs=["COE","ASE","AIY", "ADL"]


models=all_models
models=[models[4]]
model_names=['LDA','QDA','SGD','KNN','GPC','DTC','RFC','ABC','GBC','VC','MLP','LR','NB','SVM1']
model_names=[model_names[4]]


motif_set_name="individual_motifs"
print("model","m_recall","m_accur","m_roc_auc","m_ranksum_p","m_ranksum_z")
outfile=open("050919_Results_ind.csv",'w')


outfile.write("%s,%s,%s,%s,%s,%s,%s\n" %("motif_set","model","k","i","recall","ranksum_z","ranksum_p"))
#r is the random seed used to draw genes
r=0
train=import_data(motifs[0],r)
#print(train)
feature_set=getHeaders(train)

for motif in motifs:
    for i in range(len(models)):
        model=models[i]
        model_name=model_names[i]
        #print(model)
        recalls=[]
        accurs=[]
        roc_aucs=[]
        ranksum_zs=[]
        ranksum_ps=[]
        fprs=[]
        tprs=[]
        print(model_name)

        for r in range(10):
            #print(r)
            train=import_data(motif,r)
            #print(motifs[0],len(train))



            results1,results2=model_data(train,r,feature_set)

            #results1= [fpr,tpr,ranksum_z,ranksum_p,recall,accur,roc_auc]
            #results2=[fpr2,tpr2,anksum_z2,ranksum_p2,recall2,accur2,roc_auc2]
            outfile.write("%s,%s,%s,%s,%s,%s,%s\n" %(motif,model_name,r+1,1,results1[4],results1[2],results1[3]))
            outfile.write("%s,%s,%s,%s,%s,%s,%s\n" %(motif,model_name,r+1,2,results2[4],results2[2],results2[3]))

            print("%s,%s,%s,%s,%s,%s,%s\n" %(motif,model_name,r+1,1,results1[4],results1[2],results1[3]))
            print("%s,%s,%s,%s,%s,%s,%s\n" %(motif,model_name,r+1,2,results2[4],results2[2],results2[3]))
            #print(results1[4],results2[4],'recalls for 2-fold shuffle and split')

            #fpr,tpr,ranksum_z,ranksum_p,recall,accur,roc_auc=model_data_by_motifs(train,test,r,feature_set)
            #print("ranksum_z","ranksum_p","recall","accur","roc_auc")
            #print(ranksum_z,ranksum_p,recall,accur,roc_auc)
            fprs.append(fpr)
            tprs.append(tpr)
            ranksum_zs.append(ranksum_z)
            ranksum_ps.append(ranksum_p)
            recalls.append(recall)
            accurs.append(accur)
            roc_aucs.append(roc_auc)
            m_recall=sum(recalls)/len(recalls)
        print('len recalls',len(recalls))
        #get mean stats over all cross validations
        m_accur=sum(accurs)/len(accurs)
        m_roc_auc=sum(roc_aucs)/len(roc_aucs)

        m_ranksum_p=sum(ranksum_ps)/len(ranksum_ps)
        m_ranksum_z=sum(ranksum_zs)/len(ranksum_zs)

        #get median auc_roc (if list size is even, use value adjacent to median (can't use mean of medians for plotting))
        #sort auc_rocs, get value at middle,get index of value in unsorted list, get fpr list at same index
        #make copy of aucs list, then sort the copy
        sorted_roc_aucs=sorted(roc_aucs)
        mid=int((len(roc_aucs)/2))
        med_auc=sorted_roc_aucs[mid]
        i=roc_aucs.index(med_auc)
        med_fpr=fprs[i]
        med_tpr=tprs[i]
        #get standard error of the mean for results
        sem_recalls=scipy.stats.sem(recalls)
        sem_accurs=scipy.stats.sem(accurs)
        sem_roc_aucs=scipy.stats.sem(roc_aucs)
        sem_ranksum_ps=scipy.stats.sem(ranksum_ps)
        sem_ranksum_zs=scipy.stats.sem(ranksum_zs)

        print(motif_set_name,model_name,m_recall,m_accur,m_roc_auc,m_ranksum_p,m_ranksum_z,len(train))
        #print(sem_recalls,sem_accurs,sem_roc_aucs,sem_ranksum_ps,sem_ranksum_zs)
outfile.close()

model m_recall m_accur m_roc_auc m_ranksum_p m_ranksum_z


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


GPC
COE,GPC,1,1,0.833333333333,5.31195612559,6.96475154975

COE,GPC,1,2,0.677419354839,3.51301890734,3.35355123088

COE,GPC,2,1,0.666666666667,3.62502290022,3.53919857297

COE,GPC,2,2,0.909090909091,3.91708635664,4.04756768433

COE,GPC,3,1,0.692307692308,3.62451078613,3.53833802324

COE,GPC,3,2,0.758620689655,3.13562328475,2.76576289107

COE,GPC,4,1,0.571428571429,4.36049181732,4.88682423924

COE,GPC,4,2,0.888888888889,4.17529718415,4.52637102585

COE,GPC,5,1,0.769230769231,4.5854275992,5.34384595307

COE,GPC,5,2,0.689655172414,3.60765259643,3.51006988431

COE,GPC,6,1,0.769230769231,4.11339828752,4.40907253536

COE,GPC,6,2,0.620689655172,2.71416854217,2.17755499611

COE,GPC,7,1,0.769230769231,4.53485303009,5.23924882618

COE,GPC,7,2,0.724137931034,4.26512199485,4.69941972183

COE,GPC,8,1,0.777777777778,4.34365594157,4.85346601407

COE,GPC,8,2,0.785714285714,4.37732769306,4.92030054032

COE,GPC,9,1,0.692307692308,2.49501207604,1.89979202315

COE,GPC,9,2,0.51724137931,4.63600216831,5.449

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:510: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


COE,LDA,1,1,0.458333333333,0.237595481656,0.0903397657992

COE,LDA,1,2,0.58064516129,1.824393877,1.16690042063

COE,LDA,2,1,0.484848484848,0.678618031085,0.303311756197

COE,LDA,2,2,0.636363636364,1.4774974854,0.855294023934

COE,LDA,3,1,0.615384615385,1.05363685643,0.534543966728

COE,LDA,3,2,0.48275862069,-0.85976767485,0.40902769437

COE,LDA,4,1,0.25,-0.564001837375,0.242032717062

COE,LDA,4,2,0.592592592593,1.08591398539,0.556710331492

COE,LDA,5,1,0.576923076923,1.20536056376,0.641942998863

COE,LDA,5,2,0.413793103448,1.01149138218,0.506149931294

COE,LDA,6,1,0.730769230769,2.03984095406,1.38335469487

COE,LDA,6,2,0.620689655172,1.1463568998,0.599207365142

COE,LDA,7,1,0.576923076923,2.06512823861,1.40992907916

COE,LDA,7,2,0.551724137931,1.26436422772,0.685923427878

COE,LDA,8,1,0.62962962963,1.3468700594,0.749526114566

COE,LDA,8,2,0.392857142857,0.858629662869,0.408329065405

COE,LDA,9,1,0.5,0.952487718217,0.467437094426

COE,LDA,9,2,0.551724137931,-0.0927200433662,0.0333299357